In [1]:
# Importing Libraries
#!pip install tensorflow

In [2]:
#!pip install keras

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'
#DATADIR = 'gdrive/''My Drive''/UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
#'gdrive/''My Drive''/

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        #filename = f'gdrive/''My Drive''/UCI_HAR_Dataset/'+subset+'/''Inertial Signals''/'+signal+'_'+subset+'.txt'
        
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [18]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    #filename = f'gdrive/''My Drive''/UCI_HAR_Dataset/'+subset+'/y_'+subset+'.txt'
    
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [12]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [14]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [15]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [16]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [19]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [20]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [ ]:
# Initializing parameters
epochs = 30
batch_size = 64
n_hidden = 32

- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3678 - acc: 0.8021 - val_loss: 0.6611 - val_acc: 0.7377
Epoch 2/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3584 - acc: 0.7999 - val_loss: 0.5097 - val_acc: 0.7621
Epoch 3/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3399 - acc: 0.8043 - val_loss: 0.5100 - val_acc: 0.7499
Epoch 4/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3540 - acc: 0.8081 - val_loss: 0.5005 - val_acc: 0.7560
Epoch 5/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3349 - acc: 0.8100 - val_loss: 0.4837 - val_acc: 0.7618
Epoch 6/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3152 - acc: 0.8141 - val_loss: 0.4966 - val_acc: 0.7547
Epoch 7/30
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3189 - acc: 0.8211 - val_loss: 0.6462 - val_acc: 0.7377
Epoch 8

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 509        1         0        0                   0   
SITTING                  0      421        61        1                   1   
STANDING                 0      132       398        2                   0   
WALKING                  0        0         0      470                  19   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0       16                  23   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            7  
STANDING                           0  
WALKING                            7  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 432  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 12s 4ms/step


In [0]:
score

[0.3021577629208575, 0.8992195453003053]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

## 1 Hyperparameter Tuning LSTM units

**Procedure** :

1. In this model, we design a 1 hidden layer neural network with the following arcitecture

2. Architecture : 
   INPUT - LSTM(n_units) - DROPUT(0.5) - DENSE(6) - OUTPUT

3.  Number of LSTM units is a hyperparameter.
4.  To tune this, we use keras sklearn's wrapper and perform 3-fold GridSearch cross validation.

5.  With optimum LSTM units, the model is fitted with train data and finally tested.

In [25]:
from keras.initializers import he_normal
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [79]:
# Initializing parameters
epochs = 30
batch_size = 64
#n_hidden = 32

- Defining the Architecture of LSTM

In [80]:
def best_LSTM_units(n_units):
    # Initiliazing the sequential model
    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(n_units, input_shape=(timesteps, input_dim),kernel_initializer=he_normal(seed=None)))
    # Adding a dropout layer
    model.add(Dropout(0.5))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    #model.summary()
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

In [81]:
n_units = [16,32,64,128]

model = KerasClassifier(build_fn=best_LSTM_units, epochs=epochs, batch_size=batch_size, verbose=0)
param_grid = dict(n_units=n_units)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=3)
grid_result = grid.fit(X_train, Y_train,validation_data=(X_test, Y_test))

In [82]:
print("Best: score %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
print("\nScores\n")
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: score 0.925734 using {'n_units': 128}

Scores

0.718580 (0.064194) with: {'n_units': 16}
0.894587 (0.020617) with: {'n_units': 32}
0.889554 (0.029379) with: {'n_units': 64}
0.925734 (0.016083) with: {'n_units': 128}


In [83]:
from keras.utils import to_categorical

In [84]:
y_pred=to_categorical((grid_result.predict(X_test)))

In [85]:
# Confusion Matrix
print(confusion_matrix(Y_test, y_pred))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      420        70        0                   0   
STANDING                 0       90       441        1                   0   
WALKING                  0        3         2      473                  13   
WALKING_DOWNSTAIRS       0        0         0        2                 413   
WALKING_UPSTAIRS         0        1         1        4                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            5  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 464  


In [86]:
score = grid.score(X_test, Y_test)

In [88]:
print("Test Score : ",score)

Test Score :  0.9233118425517476


**Observation** 

1. The model performed well with 
   -  train score of 92.5% and 
   -  test scores of 92.3%
2. The model misclassifies 
   -  14% of SITTING classes as STANDING and 
   -  17% of STANDING classes as SITTING

# 2 Tuning Dropout Rate

**Procedure** :

1. In this model, we design a 1 hidden layer neural network with the following arcitecture

2. Architecture : 
   INPUT - LSTM(128) - DROPUT(dropout_rate) - DENSE(6) - OUTPUT

3.  We consider 128 LSTM units as tuned earlier. Here, the dropout rate is the hyper parameter.

4.  To tune Dropout rate, we use keras sklearn's wrapper and perform 3-fold GridSearch cross validation.

5.  With optimum dropout rate, we determine test accuracy of the model.

In [89]:
def best_dropout_rate(dropout_rate):
    # Initiliazing the sequential model
    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(128, input_shape=(timesteps, input_dim),kernel_initializer=he_normal(seed=None)))
    # Adding a dropout layer
    model.add(Dropout(dropout_rate))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    #model.summary()
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

In [90]:
dropout_rate = np.arange(0.1,1.0,0.2)

model = KerasClassifier(build_fn=best_dropout_rate, epochs=epochs, batch_size=batch_size, verbose=0)
param_grid = dict(dropout_rate=dropout_rate)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=3)
grid_result = grid.fit(X_train, Y_train,validation_data=(X_test, Y_test))

In [91]:
print("Best train score : %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
print("\nScores\n")
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best train score : 0.924918 using {'dropout_rate': 0.5000000000000001}

Scores

0.916621 (0.026336) with: {'dropout_rate': 0.1}
0.913629 (0.013341) with: {'dropout_rate': 0.30000000000000004}
0.924918 (0.014438) with: {'dropout_rate': 0.5000000000000001}
0.918390 (0.010504) with: {'dropout_rate': 0.7000000000000001}
0.911589 (0.020486) with: {'dropout_rate': 0.9000000000000001}


In [94]:
y_pred=to_categorical((grid_result.predict(X_test)))

In [95]:
# Confusion Matrix
print(confusion_matrix(Y_test, y_pred))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      353       120        0                   0   
STANDING                 0       74       455        1                   0   
WALKING                  0        4         0      481                   5   
WALKING_DOWNSTAIRS       0        1         0        4                 415   
WALKING_UPSTAIRS         0        7         0       11                  11   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           18  
STANDING                           2  
WALKING                            6  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 442  


In [96]:
score = grid.score(X_test, Y_test)

In [97]:
print("Test score: ",score)

Test score:  0.9012555140821175


**Observation** 

1. The model performed well with 
   -  train score of 92.4% and
   -  test score of 90.1%
2. Coming to classification matrix, the model misclassifies 
   -  26.8% of SITTING classes as STANDING and 
   -  14% of STANDING classes as SITTING

# Deep Network with two LSTM layers

**Procedure** :

1. In this model, we design a neural network with 2 hidden LSTM layers.

2. Architecture : 
   INPUT - LSTM(128) - DROPUT(0.5) - BN - LSTM(32) - DROPUT(0.5) - BN - DENSE(6) - OUTPUT

3.  All the weights are initialised using he_normal intialization.

**Network Parameters** :

1.  Optimzer - Adam

2.  loss - Categorical Cross Entropy

3.  Metrics - Accuracy

4.  Epochs  - 30

5.  Batch Size  -  64

6.  Dropout rate - 0.5

7.  LSTM units - 128 (layer 1) , 32 (layer 2)


In [116]:
from keras.initializers import he_normal
from keras.layers.normalization import BatchNormalization

In [123]:
# Initializing parameters
epochs = 30
batch_size = 64
n_hidden_1 = 128
n_hidden_2=32

In [124]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden_1,return_sequences=True,input_shape=(timesteps, input_dim),kernel_initializer=he_normal(seed=None)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(n_hidden_2,kernel_initializer=he_normal(seed=None)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid',kernel_initializer=he_normal(seed=None)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_114 (LSTM)              (None, 128, 128)          70656     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 128)          512       
_________________________________________________________________
dropout_113 (Dropout)        (None, 128, 128)          0         
_________________________________________________________________
lstm_115 (LSTM)              (None, 32)                20608     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32)                128       
_________________________________________________________________
dropout_114 (Dropout)        (None, 32)                0         
_________________________________________________________________
dense_108 (Dense)            (None, 6)                 198       
Total para

In [125]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [126]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 78s 11ms/step - loss: 1.2500 - acc: 0.6004 - val_loss: 0.9653 - val_acc: 0.6373
Epoch 2/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.9421 - acc: 0.7063 - val_loss: 0.8662 - val_acc: 0.7703
Epoch 3/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.7976 - acc: 0.7503 - val_loss: 1.0186 - val_acc: 0.6230
Epoch 4/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.6942 - acc: 0.7792 - val_loss: 0.6294 - val_acc: 0.8419
Epoch 5/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.5689 - acc: 0.8362 - val_loss: 0.5647 - val_acc: 0.8398
Epoch 6/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.4781 - acc: 0.8594 - val_loss: 0.6098 - val_acc: 0.8140
Epoch 7/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.3485 - acc: 0.9013 - val_loss: 0.3218 - val_acc: 0.8880
Epoch 

In [127]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      414        68        0                   4   
STANDING                 0       75       456        1                   0   
WALKING                  0        0         0      472                  24   
WALKING_DOWNSTAIRS       0        0         0        1                 419   
WALKING_UPSTAIRS         0        0         0       13                  31   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            5  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 427  


In [128]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 3ms/step


In [131]:
print("Test Score : ",score[1] )

Test Score :  0.9155072955548015


**Observation** 

1. The model performed well and slightly overfits with 
   -  train score of 94.9% and
   -  test score of 91.55%. 
2. Classification matrix shows that the model performed better classification than earlier models, misclassified
   -   13.8% of SITTING classes as STANDING and 
   -   14% of STANDING classes as SITTING

# CONCLUSION

In [16]:
from prettytable import PrettyTable
pretty_table = PrettyTable()
pretty_table.field_names = ["MODEL","ARCHITECTURE","HYPER PARAMETER","TRAIN ACCURACY","TEST ACCURACY"]
#Tabulating results using pretty table
pretty_table.add_row(["TUNING LSTM UNITS","IN-LSTM(units)-DO(0.5)-OUT(6)","units=128","0.9257","0.9233"])
pretty_table.add_row(["TUNING DROPOUT RATE","IN-LSTM(128)-DO(rate)-OUT(6)","rate=0.5","0.9249","0.9012"])
pretty_table.add_row(["2 HIDDEN LSTMs","IN-LSTM(128)-DO(0.5)-BN-LSTM(32)-DO(0.5)-BN-OUT(6)","-","0.9493","0.9155"])


In [17]:
print(pretty_table)
print("BN - BatchNormalization\nDO - DROPOUT\nIN - INPUT\nOUT - OUTPUT ")

+---------------------+----------------------------------------------------+-----------------+----------------+---------------+
|        MODEL        |                    ARCHITECTURE                    | HYPER PARAMETER | TRAIN ACCURACY | TEST ACCURACY |
+---------------------+----------------------------------------------------+-----------------+----------------+---------------+
|  TUNING LSTM UNITS  |           IN-LSTM(units)-DO(0.5)-OUT(6)            |    units=128    |     0.9257     |     0.9233    |
| TUNING DROPOUT RATE |            IN-LSTM(128)-DO(rate)-OUT(6)            |     rate=0.5    |     0.9249     |     0.9012    |
|    2 HIDDEN LSTMs   | IN-LSTM(128)-DO(0.5)-BN-LSTM(32)-DO(0.5)-BN-OUT(6) |        -        |     0.9493     |     0.9155    |
+---------------------+----------------------------------------------------+-----------------+----------------+---------------+
BN - BatchNormalization
DO - DROPOUT
IN - INPUT
OUT - OUTPUT 


**Conclusion**:
    
1.  All the three models gave a test score above 90%
2.  Model with 2 LSTM hidden layers classified better than the other two models with less misclassification of STANDING and SITTING classes